<a href="https://colab.research.google.com/github/AI0497/DataAnalysis/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gsutil cp -r gs://nis-interview-task-data/data-analyst/appactivity/*.parquet .

Copying gs://nis-interview-task-data/data-analyst/appactivity/part-00000-21cd72ad-bc51-48ce-b5ad-75ec555d8e62-c000.gz.parquet...
Copying gs://nis-interview-task-data/data-analyst/appactivity/part-00001-21cd72ad-bc51-48ce-b5ad-75ec555d8e62-c000.gz.parquet...
Copying gs://nis-interview-task-data/data-analyst/appactivity/part-00002-21cd72ad-bc51-48ce-b5ad-75ec555d8e62-c000.gz.parquet...
Copying gs://nis-interview-task-data/data-analyst/appactivity/part-00003-21cd72ad-bc51-48ce-b5ad-75ec555d8e62-c000.gz.parquet...
| [4 files][885.0 MiB/885.0 MiB]   37.8 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://nis-interview-task-data/data-analyst/appactivity/part-00004-21cd72ad-bc51-48ce-b5ad-75ec555d8e62-c000.gz.parquet...
Copying gs://nis-inter

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [5]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [6]:
!pip install -q findspark

In [7]:
import os

In [8]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [9]:
import findspark
findspark.init()

In [11]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
df = spark.read.parquet("/content/*.gz.parquet") 

In [30]:
# exploring the file schema
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user: string (nullable = true)
 |-- news: string (nullable = true)
 |-- event: string (nullable = true)
 |-- source: string (nullable = true)
 |-- model: string (nullable = true)
 |-- timespent: float (nullable = true)
 |-- eventdate: string (nullable = true)



In [14]:
df.registerTempTable("news")
# createTempView

In [16]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
#Metric1 : Avg Time spent per news
timeSpentPerNews=df.groupBy('news').agg({'timespent': 'mean'}).count()

In [17]:
#registering the dataFrame to a temp table
temp=sqlContext.sql("select count(*) from news")

In [ ]:
#top 20 user as per their usage
timeSpentPerUser=sqlContext.sql("select user,AVG(timespent) from news group by 1 order by 2 desc limit 20")

In [35]:
import pandas as pd
print('Data overview')
df.printSchema()
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- author: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user: string (nullable = true)
 |-- news: string (nullable = true)
 |-- event: string (nullable = true)
 |-- source: string (nullable = true)
 |-- model: string (nullable = true)
 |-- timespent: float (nullable = true)
 |-- eventdate: string (nullable = true)

Columns overview


,Column Name,Data type
0,author,string
1,timestamp,bigint
2,user,string
3,news,string
4,event,string
5,source,string
6,model,string
7,timespent,float
8,eventdate,string


In [36]:
print('Data frame describe:')
df.describe().toPandas()

Data frame describe (string and numeric columns only):


,summary,author,timestamp,user,news,event,source,model,timespent,eventdate
0,count,10870836,11039822,11039822,11039822,11039822,10870854,11039822,11039822,11039822
1,mean,None,1.6104477014813547E12,Infinity,Infinity,None,None,Infinity,15.813193783445763,None
2,stddev,None,4.1420234429519314E8,NaN,NaN,None,None,NaN,16.672127026686265,None
3,min,033e9e084df86185,1230740087161,00000ddf9b59ea7e,0002e42388eb0e95,TimeSpent-Back,016e96d11e2eb6e4,001ac13aa2b66db2,0.0,2008/12/31
4,max,feadbc41b9c20209,1639852788167,ffffeed21523023e,fffff27bb16f919f,Video Play Clicked,fee0a2920de62eb9,fff4ca8d3a83caa3,64.4,2021/12/18


In [43]:
#Finding count of missing values 
import pyspark
import numpy as np
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *
string_columns = ['author', 'user', 'news', 'event', 'source', 'model', 'eventdate']
numeric_columns = ['timestamp', 'timespent']
array_columns = []
missing_values = {} 
for index, column in enumerate(df.columns):
    if column in string_columns:    # check string columns with None and Null values
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:  # check zeroes, None, NaN
        missing_count = df.where(col(column).isin([0,None,np.nan])).count()
        missing_values.update({column:missing_count})
    if column in array_columns:  # check zeros and NaN
        missing_count = df.filter(array_contains(df[column], 0) | array_contains(df[column], np.nan)).count()
        missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,author,timestamp,user,news,event,source,model,timespent,eventdate
0,168986,0,0,0,0,168968,0,194,0


In [74]:
#top 20 trending news
trending_news=sqlContext.sql("select news,count(*) as viewCount from news group by 1 order by 2 desc limit 20")

In [76]:
#total number of count per event_date
newsCount=sqlContext.sql("select eventdate,count(*) from news group by 1 order by 2 desc")

In [78]:
#top 20 active user
mostActiveuser=sqlContext.sql("select user,sum(timespent) from news group by 1 order by 2 desc limit 20")

In [ ]:
#converting pandas df to plot histogram
pdf_newsCount=newsCount.toPandas()
pdf_trending_news=trending_news.toPandas()
pdf_mostActiveuser=mostActiveuser.toPandas()
display(pdf_newsCount.loc[2])

In [ ]:
pdf_newsCount.plot(kind='bar',x='news',y='count(1)',colormap='plasma')
pdf_trending_news.plot(kind='pie',x='news',y='count(1)',colormap='plasma')
pdf_mostActiveuser.plot(kind='barh',x='news',y='count(1)',colormap='plasma')
